In [ ]:
import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt

hv.extension('matplotlib')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
c = 299792458 # m/c
mc = 0.511e6 # eV/c
Qe = 1.60217662e-19 # elementary charge in Coulombs

#p0 = 393e6 # eV/c
p0=600e6
#p0=2000e6

In [ ]:
Ne = 3.7e11  # Number of electrons/positrons in the beam
N  = 1000# number of macro-particles in this simulation

In [ ]:
print("Bunch charge = %.1f nC" % (Ne*Qe/1e-9))

Electron beam parameters:

In [ ]:
#sigma_z = 0.22 # m for 393 MeV
sigma_z=0.29 # for 600 Mev
#sigma_z= 0.091 # for 2 GeV
#sigma_z = 1.0e-2 # m -- to test wakefield calculation

#sigma_dp = 1.25e-4 # relative momentum spread
#sigma_dp=0.000135638 # for 393 MeV
sigma_dp=0.000210 # for 600 MeV
#sigma_dp=0.00072 # for 2 Gev

Distribution in $z$ and $\delta p = \frac{\Delta p}{p}$ can be defined easily since they are not correlated:

In [ ]:
z0  = np.random.normal(scale=sigma_z, size=N)
#z0  = np.random.uniform(low=-sigma_z*2, high=sigma_z*2, size=N)
dp0 = np.random.normal(scale=sigma_dp, size=N)

In [ ]:
%opts Scatter (alpha=1 s=2) [aspect=3 show_grid=True]

dim_z  = hv.Dimension('z',  unit='m', range=(-12,12))
dim_dp = hv.Dimension('dp', label='100%*$\Delta p/p$', range=(-1.5,+1.5))

%output backend='matplotlib' fig='png' size=200 dpi=100

In [ ]:
hv.Scatter((z0,dp0*100), kdims=[dim_z,dim_dp])



The function to get beam current profile corresponding to particle distribution:

In [ ]:
def get_I(z, z_bin = 0.05, z_min=-15, z_max=+15):
    # z, z_bin, z_min, z_max in meters
    
    hist, bins = np.histogram( z, range=(z_min, z_max), bins=int((z_max-z_min)/z_bin) )
    Qm = Qe*Ne/N # macroparticle charge in C
    I = hist*Qm/(z_bin/c) # A

    z_centers = (bins[:-1] + bins[1:]) / 2
    
    return z_centers, I

In [ ]:
%opts Area [show_grid=True aspect=3] (alpha=0.5)

dim_I = hv.Dimension('I',  unit='A',  range=(0.0,+6.0))

hv.Area(get_I(z0), kdims=[dim_z], vdims=[dim_I])
#plt.scatter(get_I(z0)[0],get_I(z0)[1])

### Effects of RF-resonator

Longitudinal momentum gain of electron after it has passed through the RF-resonator depends on the electron phase with respect to the RF:

$$
\frac{dp_z}{dt} = eE_{\rm{RF}}\cos\phi,
$$

where $E_{\rm{RF}}$ is the accelerating electric field and $\phi$ is the electron phase in the RF resonator. The resulting longitudinal momentum change:

$$
\delta p_z = e\frac{ V_{\rm{RF}} }{ L_{\rm{RF}}} (\cos\phi) \Delta t = e\frac{ V_{\rm{RF}} }{c} \cos\phi,
$$

where $V_{\rm{RF}}$ is the RF-voltage.

RF-resonator frequency $f_{\rm{RF}}$ is some harmonic $h$ of revolution frequency:

$$
f_{\rm{RF}} = \frac{h}{T_s},
$$

where $T_s$ is the revolution period.

Longitudinal coordinate $z$ gives the longitudinal distance from the electron to the reference particle at the moment when the reference particle arrives at the RF-phase $\phi_0$ (which is always the same). So the electron then arrives to the RF-resonator after the time

$$
\Delta T = -\frac{z}{c}.
$$

Then the electron phase in the RF-resonator is

$$
\phi = \phi_0 + 2\pi f_{\rm{RF}}\Delta T = \phi_0 - 2\pi \frac{hz}{T_s c} \approx \phi_0 - 2\pi \frac{hz}{L}.
$$

where $L$ is the ring perimeter. If the electron momentum is different from its reference value then the period of revolution $T$ is different from $T_s$:
$$
T = \frac{L+\Delta l}{\upsilon_s + \Delta \upsilon} \approx T_s \left ( 1 + \frac{\Delta l}{L} - \frac{\Delta \upsilon}{\upsilon_s} \right ) \approx T_s \left ( 1 + \frac{\Delta l}{L} - \frac{1}{\gamma^2} \frac{\Delta p}{p} \right ).
$$
The difference between the length of electron trajectory and the reference orbit length is given by the $M_{56}$ element of the 1-turn transport matrix:
$$
\Delta l = M_{56} \frac{\Delta p} {p}.
$$
$\Delta T = T - T_s$ in can be written as
$$
\Delta T \approx T_s \left ( \frac{M_{56}}{L} - \frac{1}{\gamma^2} \right ) \frac{\Delta p}{p} = T_s \left ( \frac{1}{\gamma_t^2} - \frac{1}{\gamma^2} \right ) \frac{\Delta p}{p} = T_s\eta\frac{\Delta p}{p}.
$$

Then the longitudinal position after one turn is
$$
z_{n+1} = z_n - L \eta\frac{\Delta p_{n+1}}{p}.
$$

## Multi-turn tracking

In [ ]:
L = 74.39 # m -- VEPP-3 perimeter
gamma_t = 3.752 # gamma transition in the VEPP-3
eta = 1/(gamma_t*gamma_t) - 1/((p0/mc)*(p0/mc))

In [ ]:
#N_turns = 1000020
N_turns = 10000
N_plots = 11

h = 2
#h=18
#eVrf = 20e3 # eV
#eVrf=800e3 
eVrf = 9.7e3 # eV
phi0 = np.pi/2
#phi0 = 3.05258 # VEPP-3 equilibrum synchrotron phase 174.9 deg 
#dpoteri=1.81e3/p0
w_synch_1=1770*2*np.pi
w_synch_2=26700*2*np.pi
w_rev=4030000
t_plots = np.arange(0,N_turns+1,int(N_turns/(N_plots-1)))

data2plot = {}
#data2plot_2 = {}

z = z0; dp = dp0
phi=np.random.normal(scale=phi0, size=N)
for turn in range(0,N_turns+1):
    if turn in t_plots:
        print( "\rturn = %g (%g %%)" % (turn, (100*turn/N_turns)), end="")
        data2plot[turn] = (z,dp)
        #data2plot_2[turn] = (phi,dp)
    
    phi = phi0 - 2*np.pi*h*(z/L)  # phase in the resonator
    
    # 1-turn transformation:
    dp  = dp + eVrf*np.cos(phi)/p0
    z = z - L*eta*dp
    

In [ ]:
#phase=np.linspace(-10*np.pi,10*np.pi,60)
#W_bucket=L*(eVrf*p0/(2*np.pi*h*eta))**(0.5)/(np.pi*h*c)
# separatrix up/down
#s_u=W_bucket*(np.cos(0)**2-np.cos(phase/2)**2)**(0.5)
#s_d=-W_bucket*(np.cos(0)**2-np.cos(phase/2)**2)**(0.5)

In [ ]:
one_particle_z=[]
one_particle_p=[]
def plot_z_dp(turn):
    z, dp = data2plot[turn]
    #phi, dp = data2plot_2[turn]
    z_dp = hv.Scatter((z, dp*100), [dim_z,dim_dp])
    #z_dp = hv.Scatter((phi, dp*100), [dim_z,dim_dp])
    z_I  = hv.Area(get_I(z), kdims=[dim_z], vdims=[dim_I])
    #sep_1 = hv.Curve((phase, s_u), kdims=[dim_z], vdims=[dim_dp])
    #sep_2 = hv.Curve((phase, s_d), kdims=[dim_z], vdims=[dim_dp])
    one_particle_z.append(z[1])
    one_particle_p.append(dp[1])
    return (z_dp+z_I).cols(1) 

In [ ]:
#plot_z_dp(1000)

In [ ]:
dim_z  = hv.Dimension('z',  unit='m', range=(-10,10))
dim_dp = hv.Dimension('dp', label='100%*$\Delta p/p$', range=(-0.5,+0.5))
items = [(turn, plot_z_dp(turn)) for turn in t_plots]
#items = [(turn, plot_z_dp(turn)) for turn in range(0,20000)]

m = hv.HoloMap(items, kdims = ['Turn'])
m.collate()

In [ ]:
#hv.Scatter((one_particle_z,one_particle_p))
plt.scatter(one_particle_z,one_particle_p,s=1)
plt.ylim(-0.001,0.001)

## Longitudinal wakefield

Now let's introduce the longitiudinal wakefield.

First define wake-function in terms of $\xi = z - ct$:

Equation for wake-function can be obtained by performing a Fourier transformation of the impedance

$$
Z = \frac{R_s}{1 + iQ\left(\frac{\omega_R}{\omega} -\frac{\omega}{\omega_R}\right)},
$$
where $Q$ is the quality factor, $\omega_R$ is the frequency.

(from A. Chao's book &#147;<a href=http://www.slac.stanford.edu/~achao/wileybook.html>Physics of Collective Beam Instabilities in High Energy Accelerators</a>&#148;. <a href=http://www.slac.stanford.edu/~achao/WileyBook/WileyChapter2.pdf>Chapter 2</a>, p. 73):

$$
W(\xi) = \begin{cases}
 2\alpha R_s e^{\alpha \xi/c}\left(\cos\frac{\overline{\omega} \xi}{c} + \frac{\alpha}{\overline{\omega}}\sin\frac{\overline{\omega} \xi}{c}\right), & \mbox{if } \xi < 0 \\
 \alpha R_s, & \mbox{if } \xi = 0 \\
 0, & \mbox{if } \xi > 0,
\end{cases}
$$

where $\alpha = \omega_R / 2Q$ and $\overline\omega = \sqrt{\omega_R^2 -\alpha^2}$.

In [ ]:
def Wake(xi):
    # of course some other wake can be defined here.
    
    #fr = 0.30e9 # Hz
    #Rs=5.5e4 # Ohm
    
    fr=1.6e8 # for sigma_z=0.3m, Q=1 (VEPP-4)
    Rs = 252
    
    #fr=1.36e9 # for normal working
    #Rs=1.0e4
    
    
    #fr=100e7 # for Q=10000
    #Rs=6.4e6
    
    Q  = 1 # quality factor
   
    wr = 2*np.pi*fr
    alpha = wr/(2*Q)
    wr1 = wr*np.sqrt(1 - 1/(4*Q*Q))
    
    W = 2*alpha*Rs*np.exp(alpha*xi/c)*(np.cos(wr1*xi/c) + (alpha/wr1)*np.sin(wr1*xi/c))
    W[xi==0] = alpha*Rs
    W[xi>0] = 0
    
    return W

In [ ]:
def Wake_RF(xi):
    
    fr=72.54e6 # for Q=10000 
    #fr=1e9
    Rs=6.4e6
    Q  = 10000 # quality factor
    
    wr = 2*np.pi*fr
    alpha = wr/(2*Q)
    wr1 = wr*np.sqrt(1 - 1/(4*Q*Q))
    
    W = 2*alpha*Rs*np.exp(alpha*xi/c)*(np.cos(wr1*xi/c) + (alpha/wr1)*np.sin(wr1*xi/c))
    W[xi==0] = alpha*Rs
    W[xi>0] = 0
    
    return W

In [ ]:
%opts Curve [show_grid=True aspect=3]

dim_xi   = hv.Dimension('xi', label=r"$\xi$", unit='m',range=(-10,10))
dim_Wake = hv.Dimension('W',  label=r"$W$", unit='V/pC',range=(-10,+30))
dim_Wake_RF = hv.Dimension('W_RF',  label=r"$W_RF$", unit='V/pC',range=(-10,+30))

L_wake = 10 # m
dz = 0.04 # m 
#dz=0.001
xi = np.linspace(-L_wake, 0, 250) # m
print(xi)
W = Wake(xi)
W_RF = Wake_RF(xi)

(hv.Curve((xi, W/1.0e13), kdims=[dim_xi], vdims=[dim_Wake]) + \
 hv.Curve((xi, W_RF/1.0e12), kdims=[dim_xi], vdims=[dim_Wake_RF])).cols(1)

### Wakefield from e-bunch

Longitudinal wake-function defines the wakefield amplitude from a point-like charge. Therefore a distribution of charge will produce the wakefield

$$
E(z) = -\int\limits_{z}^{+\infty} W(z-z')I(z')dz'/c = -\int\limits_{-\infty}^{0} W(\xi)I(z-\xi)d\xi/c,
$$

In [ ]:
zc, I = get_I(z0, z_bin=dz)

V = -np.convolve(W, I)*dz/c # V
V_RF=np.convolve(W_RF,I)*dz/c

print(len(W),len(I),len(V))

In [ ]:
zV = np.linspace(max(zc)-dz*len(V), max(zc), len(V))

In [ ]:
dim_V = hv.Dimension('V', unit='kV', range=(-30,30))

(hv.Curve((zV, V/1e2), kdims=[dim_z], vdims=[dim_V]) + \
 hv.Area((zc,I), kdims=[dim_z], vdims=[dim_I])).cols(1)

## Tracking with impedance

In [ ]:
data2plot = {}
data2plot_2={}

N_turns = 20001
#eVrf = 0    # V
#eVrf = 3e3 # V
Tradz=0.08446243 # for 600 MeV
#Tradz=0.3005654 # for 393 MeV

t_plots=[i for i in range(0,20001,1000)]
#t_plots=[i for i in range(0,20000,1000)]
z = z0; dp = dp0
phi0=0 #0.19 with dpoteri
phi= np.random.normal(scale=2, size=N)
for turn in range(0,N_turns+1):
    if turn<10001:
        h=2
        eVrf=9.7e3 # for 600 Mev
        #eVrf=4.7e3 # for 393 Mev
        w_s=2*np.pi*2419.6
        #dpoteri=0.33e3/p0 # for 393 Mev
        dpoteri=8.4e3/p0
        #dpoteri=1.81e3/p0
        #beta=w_rev/(Tradz*w_s)
    else:
        h=18
        eVrf=240e3
        w_s=2*np.pi*36106.22092
        phi0=0 #0.01 with dpoteri
        #dpoteri=1.81e3/p0 
        dpoteri=0
        #beta=1/(Tradz)
    if turn in t_plots:
        print( "\rturn = %g (%g %%)" % (turn, (100*turn/N_turns)), end="")
        data2plot[turn] = (z,dp)
        data2plot_2[turn] = (phi,dp)
    
    phi = phi0 + 2*np.pi*h*(z/L)  # phase in the resonator
    
    #z = z - L*eta*dp
    #sigma_momentum=np.std(dp)
    
    # wakefield:
    zc, I = get_I(z, z_bin=dz) # A
    V = -np.convolve(W, I)*dz/c # V    
    V_RF=-np.convolve(W_RF, I)*dz/c
    V_s = np.interp(z,zV,V)
    V_s_RF=np.interp(z,zV,V_RF)
    dp = dp + V_s/p0
    dp = dp + V_s_RF/p0
    #if turn>20000:
     #   dp = dp + V_s_RF/p0
    
    # RF-cavity
    #dp=dp-dpoteri
    dp  = dp + eVrf*np.sin(phi)/p0
    
    # radiation damping and quantum excitation
    #Tradz=0.08446243
    beta=1/(Tradz)
    #dp=dp-2*beta*dp/w_rev+(12*beta/w_rev)**(0.5)*np.random.uniform(-1,1,len(dp))*sigma_dp
    dp=dp-2*beta/w_s*dp+(12*beta/w_s)**(0.5)*np.random.uniform(-1,1,len(dp))*sigma_dp
    
    # z after one turn:
    z = z - L*eta*dp

In [ ]:
def plot_z_dp(turn):
    z, dp = data2plot[turn]
    phi, dp = data2plot_2[turn]
    if max(phi)>20:
        out.append(turn)
    z1=[]
    dp1=[]
    for i in range(0,len(z)):
        if abs(z[i])<2:
            z1.append(z[i])
            dp1.append(dp[i])
    Nparticles.append(len(z1))
    standart_deviation_z.append(np.std(z1))
    standart_deviation_p.append(np.std(dp1))
    synchotronous_phase.append(np.mean(phi))
    z_dp = hv.Scatter((z, dp*100), [dim_z,dim_dp])
    #z_dp = hv.Scatter((phi, dp*100), [dim_phi,dim_dp])
    #z_dp=plt.scatter(z,dp,s=1)
    zc, I = get_I(z, z_bin=dz)
    if turn==130:
        for i in I:
            dat.append(i)
    z_I  = hv.Area((zc,I), kdims=[dim_z], vdims=[dim_I])
    V = -np.convolve(W, I)*dz/c # V
    z_V  = hv.Curve((zV, V/1e3), kdims=[dim_z], vdims=[dim_V])
    return (z_dp+z_I+z_V).cols(1)
    #return z_dp
      

In [ ]:
%opts Scatter (alpha=1 s=2) [aspect=3 show_grid=True]
%opts Area [show_grid=True aspect=3] (alpha=0.5)
dat=[]
standart_deviation_z=[]
standart_deviation_p=[]
synchotronous_phase=[]
Nparticles=[]
out=[]

#t_plots=[i for i in range(0,501,10)]
dim_z  = hv.Dimension('z',  unit='m', range=(-10.5,10.5))
dim_phi  = hv.Dimension('phi',  unit='rad', range=(-9.28,14.28))
dim_dp = hv.Dimension('dp', label='100*$\Delta p/p$', range=(-2.5,+2.5))
dim_I = hv.Dimension('I',  unit='усл.ед.',  range=(0.0,+15))
items = [(turn, plot_z_dp(turn)) for turn in t_plots]
#plt.ylim(-0.004,0.004)
#plt.xlim(-0.3,0.3)


print("synchotronous_phase",synchotronous_phase)
print("min/max",min(phi),max(phi))
print("particle is out at turn number",out)
print("N particles stable",Nparticles)
m = hv.HoloMap(items, kdims = ['Turn'])
m.collate()


In [ ]:
#np.save("‪data.npy", standart_deviation_z)

In [ ]:
plt.plot(t_plots,standart_deviation_z)

In [ ]:
plt.plot(t_plots,standart_deviation_p)

In [ ]:
f=open(r'C:\Users\Расим\Desktop\text.txt','w')
#dat=standart_deviation_z
#dat=dat[0]
#dat1=data2plot[13300]
#np.savetxt(f,freq_2)
f.close()

In [ ]:
c = 299792458
four=abs(np.fft.fft(W_RF))
freq=np.fft.fftfreq(len(four))
freq=np.linspace(0,1,250)
freq_2=abs(freq*250/(10/c)/1e9)
for i in range(0,len(four)):
    if four[i]==max(four):
        print("frequency of wake_RF:",200.0e7*10/c/249)
        print("frequency of wake_RF in GHz:",freq_2[i]/1e9)
        print("frequency from Fourier:",freq[i], freq[len(four)-i])
        #print(30.54e11/c*np.sqrt(1-1/10e8))
        #print(30.54e11/c/249*np.sqrt(1-1/10e8))
        break
plt.plot(freq_2,four)
#plt.xlim(0e9,0.3e9)

In [ ]:
c = 299792458
print(dat)
four=abs(np.fft.fft(dat))
freq=np.fft.fftfreq(len(dat))
freq=np.linspace(0,1,750)
freq_2=abs(freq*749/(30/c)/1e9)
for i in range(0,len(dat)):
    if four[i]>20 and four[i]<30:
        #print("frequency of wake_RF:",72.54e7*30/c/749)
        #print("frequency of wake_RF in GHz:",freq_2[i]/1e9)
        #print("frequency from Fourier:",freq[i], freq[len(four)-i])
        #print(30.54e11/c*np.sqrt(1-1/10e8))
        #print(30.54e11/c/249*np.sqrt(1-1/10e8))
        break
plt.plot(freq_2,four)
plt.xlim(0,2)
plt.ylim(0,80)
print("frequency of wake_RF:",100.54e7*30/c/749)
#print("frequency of wake_RF in GHz:",freq_2)


In [ ]:
f=open(r'C:\Users\Расим\Desktop\testing.txt','w')
#dat=standart_deviation_z
#dat=dat[0]
#dat1=data2plot[13300]
np.savetxt(f,four)
f.close()